# Creating Vector Database 

In [5]:
#initialize chromadb collection
import chromadb

client = chromadb.PersistentClient(path="C:/Users/User/Documents/SideProject/personal_document_chatbot_with_RAG/data/vectorDB")

collection = client.create_collection(name="document_collection")

UniqueConstraintError: Collection document_collection already exists

In [2]:
import sentence_transformers
model = sentence_transformers.SentenceTransformer("mixedbread-ai/mxbai-embed-large-v1")

c:\Users\User\Documents\SideProject\personal_document_chatbot_with_RAG\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [13]:
import json 
json_path = "C:/Users/User/Documents/SideProject/personal_document_chatbot_with_RAG/scripts/processed_data.json"

#open json file
with open(json_path, 'r', encoding='UTF-8') as file:
    data = json.load(file)


for entries in data:
    chunk_id = entries.get("chunk_id")
    content = entries.get("content")
    # metadata stores the heading, source and tokens info in a dict
    metadata = {k:v for k, v in entries.items() if k not in ["chunk_id", "content"]} # get the key and values for each entries as long as the key is not chunk_id or content
    
    
    # generate embedding of content for each entries
    content_embedding = model.encode(content)
    print(content_embedding)



[ 0.20839214 -0.5843315  -0.38129288 ... -0.4346542   0.36485964
  0.16328812]
[-0.43508515  0.06438106 -0.30861503 ...  0.3251494  -0.4378202
 -0.02801888]
[-0.26943856  0.3203061  -0.5620354  ...  0.09387776 -0.25365424
 -0.9855117 ]
[ 0.3125392   0.27092224 -0.05939057 ...  0.29246563 -0.24291721
 -0.6209721 ]
[-0.4235745  -0.7117383   0.01221112 ... -0.1778752  -1.0335604
  0.3561757 ]
[ 0.1128249  -0.4829722   0.01347639 ...  0.14912443 -0.02831652
 -0.38028228]
[ 0.00654855 -0.40538773 -0.74450153 ... -0.85515803 -0.43230224
 -0.06256361]
[ 0.3517173   0.08681054  0.07308173 ...  0.16460116 -0.36276403
  0.7847017 ]
[ 0.5134605   0.20791458 -0.32867682 ...  0.39174056 -0.13730437
  0.35328588]
[ 0.38418445 -0.4348011  -0.7738068  ... -0.1806384  -0.5820453
  0.9350648 ]
[ 0.30216134 -0.23373216 -0.12324968 ... -0.53406066 -0.09703822
 -0.32361585]
[-0.17486848  0.03676977 -0.4087963  ... -0.5527557  -0.14614144
 -0.2080621 ]
[ 0.35138243 -0.12034628  0.11571164 ... -0.50182176 -0

In [ ]:
embedding = model.encode("K231189695108 ** T231175999163 (Single)")

# Check the type and shape
print(embedding)
print(type(embedding))  # Expected: <class 'numpy.ndarray'>
print(embedding.dtype)  # Expected: float32 data type
print(embedding.shape)  # Expected: (1024,) Shape is the number of items along the axis (row = 1024 items, column)
print(embedding.ndim)   # Dimension is the number of axes in the array (1D, 2D, 3D)


""" IMPORTANT TO REMEMBER:

    PYTHON LISTS DONT SUPOORT NUMPY METHODS

"""

[-0.43508515  0.06438106 -0.30861503 ...  0.3251494  -0.4378202
 -0.02801888]
<class 'numpy.ndarray'>
float32
(1024,)
1


In [41]:
# Adding information from .json file into the chromaDB collection
import json
from sentence_transformers import SentenceTransformer
import numpy as np

#Step 1: load JSON data
json_path = "C:/Users/User/Documents/SideProject/personal_document_chatbot_with_RAG/scripts/processed_data.json"
with open(json_path, 'r', encoding='UTF-8') as file:
    data = json.load(file)

#Step 2: load embedding model (mxbai-embed-large/e5-large-v2/stella_400m)
model = SentenceTransformer("mixedbread-ai/mxbai-embed-large-v1")

#Step 3: insert data into ChromaDB
    #Step 3.1: loop every entries in the json 
for entries in data:
    chunk_id = entries.get("chunk_id")
    content = entries.get("content"),
    metadata = {k:v for k, v in entries.items() if k not in ["chunk_id", "content"]} # get the key and values for each entries as long as the key is not chunk_id or content

    #Step 3.2: generate embedding 
    content_embedding = model.encode(content)
    #convert from ndArray to python list
    #Ensure embedding is 1D and convert to a list
    if isinstance(content_embedding, np.ndarray):
        embedding = content_embedding.flatten().tolist() 
    
    if isinstance(content, tuple):
        content = content[0]  # Extract first element if it's a tuple
    #Step 3.3: Add to chromaDB collection
    #chromadb can store the raw documents, metadata, ids, and embeddings
    collection.add(
        ids = [chunk_id],
        embeddings= [embedding],
        metadatas= [metadata],
        documents=[content]
    )

    

In [7]:

query = "CGPA"
query_embedding = model.encode(query)

print(query_embedding)

[-0.01359718 -0.5008308  -0.12128836 ... -0.18291457 -0.23046495
  0.13400184]


In [8]:
#initialize chromadb collection
import chromadb

client = chromadb.PersistentClient(path="C:/Users/User/Documents/SideProject/personal_document_chatbot_with_RAG/data/vectorDB")
collection = client.get_collection(name="document_collection")
collection.query(
    query_embeddings= query_embedding.tolist(),
    n_results=2,
    include = ["documents"]
)

{'ids': [['54169f46-1ccb-4155-ac9b-85e7aae8898a',
   'cc9ef793-d5aa-415e-ac69-c9e754f111e6']],
 'embeddings': None,
 'documents': [['Current CGPA 3.49',
   'CGPA 3.46 Git\nfreeCodeCamp Machine Learning\nSeptember 2021 Computer Vision\nMobile Development Workshop: Flutter Python\nFlask\nUITM Kampus Kuala Terengganu\nMongoDB\nOctober 2023\nFundamental of Cloud Computing\nRuncloud Education\nAugust 2022\nUtilized Tailwind, Flask, MongoDB technologies to build the web app.\nAugust 2020 - May 2021\nMarch 2024 - Present\nMachine Learning\nDec 2023 - Jan 2024\nComputer Vision\nPython\nFlask\nMongoDB\nIntermediate\n|CERTIFICATIONS CGPA 3.46 freeCodeCamp September 2021 Mobile Development Workshop: Flutter UITM Kampus Kuala Terengganu October 2023 Fundamental of Cloud Computing Runcloud Education|SKILLS Git Computer Visin Machine Learning Python Computer Vision Flask Python MongoDB Flask Intermediate MongoDB Beginner|\n|---|---|']],
 'uris': None,
 'data': None,
 'metadatas': None,
 'distances':

In [43]:
# trying to query the chroma db 


collection.get(
    ids = "01b171ea-76a7-49d5-800e-5affc2cbb65d" ,
    include=["documents"]
)

{'ids': ['01b171ea-76a7-49d5-800e-5affc2cbb65d'],
 'embeddings': None,
 'documents': ['Airasia Ride Promo Code : KTMRIDE\nSila berada di stesen 30 minit sebelum tren berlepas. Pintu pelepasan akan ditutup 5 minit sebelum waktu berlepas.\nTertakluk kepada Akta Pengangkutan Awam Darat 2010 dan  Syarat Pengangkutan Penumpang.\nKTMB CALL CENTER +603-2267 1200'],
 'uris': None,
 'data': None,
 'metadatas': None,
 'included': [<IncludeEnum.documents: 'documents'>]}